In [12]:
import pandas as pd
import cx_Oracle
import requests
from xml.etree.ElementTree import fromstring

In [13]:
access_key = 'pnWEl7Muj6PpSyrC35SaqRjYnKAAL0xedWtDdUQDmfsWvqDxK1ORAkSg2/axQsJNCKZjQlf98A3TRAuuIzJ0GQ=='
TOTAL_NUM = 102
MAX_TRANSACTION = 30
request_total_num = ( TOTAL_NUM // MAX_TRANSACTION) + 1

In [14]:
con = cx_Oracle.connect('open_source/1111@localhost:1521/xe')

In [15]:
cur = con.cursor()

In [16]:
def xml_to_oracle(data_root, data_name):
    for row in data_root.iter(data_name):
        sql_insert = '''
            insert into covid_hospital(기관명, 구분코드, 시도명, 시군구명, 전화번호, 운영가능일자)
            values(:기관명, :구분코드, :시도명, :시군구명, :전화번호, :운영가능일자)
        '''
#  'adtFrDd': '운영가능일자', 'sgguNm': '시군구명', 'sidoNm': '시도명', 'spclAdmTyCd': '구분코드', 'telno': '전화번호', 'yadmNm': '기관명'
        기관명 = row.find('yadmNm').text
        구분코드 = row.find('spclAdmTyCd').text
        시도명 = row.find('sidoNm').text
        시군구명 = row.find('sgguNm').text
        전화번호 = row.find('telno').text
        운영가능일자 = row.find('adtFrDd').text
        
        cur.execute(sql_insert, (기관명, 구분코드, 시도명, 시군구명, 전화번호, 운영가능일자))
        con.commit()

In [17]:
def get_request_url(page_no):
    url = 'http://apis.data.go.kr/B551182/pubReliefHospService/getpubReliefHospList'
    # params = {'serviceKey': access_key}
    params = {'serviceKey': access_key,'numOfRows': MAX_TRANSACTION,'pageNo': page_no}
    response = requests.get(url, params=params)
    return response.text

In [24]:
raw_xml = get_request_url(page_no)

In [25]:
root = fromstring(raw_xml)

In [26]:
data_root = root.find('body').find('items')

In [27]:
rename_dict = {'adtFrDd': '운영가능일자', 'sgguNm': '시군구명', 'sidoNm': '시도명', 'spclAdmTyCd': '구분코드', 'telno': '전화번호', 'yadmNm': '기관명'}
REDEFINE_COLUMNS = ['기관명', '구분코드', '시도명', '시군구명', '전화번호', '운영가능일자']

In [30]:
for page_no in range(1, request_total_num+1):
    raw_xml = get_request_url(page_no)
    root = fromstring(raw_xml)

    data_root = root.find('body').find('items')
    
    xml_to_oracle(data_root, 'item')

In [38]:
xml_to_oracle(data_root,'item')